In [1]:
%matplotlib inline

In [2]:
# Import Dependencies

import logging
import matplotlib.pyplot as plt

# qcodes imports
import qcodes as qc
from qcodes.dataset.experiment_container import new_experiment, load_experiment_by_name
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_by_id
# import qcodes.instrument_drivers.Keysight.N5245A as N5245A
from qcodes.instrument_drivers.Keysight.Keysight_E5071C import Keysight_E5071C

#setup
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Start experiment
exp_name = 'PNA_Example'
sample_name = 'Thru_Coax'
try:
    exp = load_experiment_by_name(exp_name, sample=sample_name)
    print('Experiment loaded. Last ID no:', exp.last_counter)
except ValueError:
    exp = new_experiment(exp_name, sample_name)
    print('Starting new experiment.')

Experiment loaded. Last ID no: 0


In [4]:
# pna = N5245A.N5245A('pna', 'TCPIP::192.168.0.10::inst0::INSTR')
vna_address='169.254.71.72'
pna = Keysight_E5071C('VNA','TCPIP0::'+vna_address+'::INSTR')

C:\Users\Public\Anaconda\lib\site-packages\qcodes\instrument\parameter.py:274: UserWarning: Wrapping get method of parameter: trace, original get method will not be directly accessible. It is recommended to define get_raw in your subclass instead. Overwriting get will be an error in the future.
  warnings.warn(f'Wrapping get method of parameter: {self.full_name},'


Connected to: Agilent Technologies E5071C (serial:MY46526831, firmware:B.13.20) in 0.41s


In [10]:
# Set up a frequency sweep from 100 MHz to 1 GHz, with 1001 points, at a power of -40dBm
pna.power(-40)
pna.start(100e6)
pna.stop(1e9)
pna.npts(1001)
pna.measure("S21")

# Enable 2 averages, and set IF BW to 1kHz
pna.bandwidth(1e3)
# pna.averages_enabled(True)
# pna.averages(2)

# Run a measurement
meas = Measurement()
meas.parameter(pna.magnitude)

with meas.run() as datasaver:
    mag = pna.magnitude()
    datasaver.add_result((pna.magnitude, mag))
    dataid = datasaver.run_id
plot_by_id(dataid)

# Other valid parameter types are:
# pna.linear_magnitude()
# pna.phase()
# pna.unwrapped_phase()
# pna.group_delay()
# pna.real()
# pna.imaginary()

AttributeError: 'Measurement' object has no attribute 'parameter'

In [12]:
help(pna)

Help on Keysight_E5071C in module qcodes.instrument_drivers.Keysight.Keysight_E5071C object:

class Keysight_E5071C(qcodes.instrument.visa.VisaInstrument)
 |  qcodes driver for the Rohde & Schwarz ZNB20 virtual network analyser
 |  
 |  Requires FrequencySweep parameter for taking a trace
 |  
 |  TODO:
 |  - centre/span settable for frequwncy sweep
 |  - check initialisation settings and test functions
 |  - S11/S12 ... selection
 |  - set correct timeout
 |  
 |  Method resolution order:
 |      Keysight_E5071C
 |      qcodes.instrument.visa.VisaInstrument
 |      qcodes.instrument.base.Instrument
 |      qcodes.instrument.base.InstrumentBase
 |      qcodes.utils.metadata.Metadatable
 |      qcodes.utils.helpers.DelegateAttributes
 |      qcodes.instrument.base.AbstractInstrument
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name, address, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  initial